# Install Simulaqron
Run the two cells below (go to cell and press SHIFT+ENTER).

In [ ]:
!pip install service_identity

In [ ]:
!pip install simulaqron

# Start virtual quantum hardware
Run the cell below (SHIFT+ENTER). You have to do this **once**, at the beginning.

If it doesn't complete within 30 secs, manually stop the cell and continue with the cells afterwards.

In [ ]:
!simulaqron start --nodes Alice,Bob,Eve --force

# Troubleshooting
You can skip this section now, come back later if any issues.
1. Did you try the **cleanup_qubits(...)** command for the misbehaving person(s)?
2. If it still doesn't work, in 'Runtime' menu select **Factory reset runtime**, then start from the beginning (i.e. from the `!pip install ...` cells above).

# Alice creates a qubit

In [ ]:
from cqc.pythonLib import *

In [ ]:
#
# Interface for Alice.
#
alice = CQCConnection("Alice")

In [ ]:
#
# Helper function to see how many qubits Alice, Bob, or Eve has in his/her quantum register.
#
def num_qubits(person):
    reg = person.active_qubits
    n = len(reg)
    print("{} has {} qubit(s).".format(person.name, n))

In [ ]:
#
# Helper function to clean up Alice's, Bob's, or Eve's quantum register.
#
def cleanup_qubits(person):
    reg = person.active_qubits
    n = len(reg)
    for i in range(n):
        q = reg[0]
        m = q.measure() # removes q from reg; we are not interested in m
        print("Measured and discarded qubit {}".format(i+1))
    print("{}'s register is empty.".format(person.name))

In [ ]:
#
# Alice creates a |0> qubit.
#
q = qubit(alice) # creates |0> by default

In [ ]:
#
# Let's see how many qubits Alice has in her quantum register.
#
num_qubits(alice)

In [ ]:
#
# Alice measures q. What result do you expect?
#
m = q.measure()
print("Measurement returned {}".format(m))

In [ ]:
#
# In SimulaQron, measurement discards the qubit, and removes it from the quantum register.
# Due to measurement, the qubit's state collapses, so it's considered "used up" by SimulaQron.
#
num_qubits(alice)

In [ ]:
#
# Let now Alice create a |1> qubit!
#
q = qubit(alice) # creates |0> by default
q.X() # the Pauli X gate: X|0>=|1>

In [ ]:
#
# Alice measures q. What result do you expect?
#
m = q.measure()
print("Measurement returned {}".format(m))

In [ ]:
#
# Again, the qubit is gone, due to measurement.
#
num_qubits(alice)

# Alice applies Hadamard gate
- Reminder:
> $H|0\rangle=|+\rangle=\frac{1}{\sqrt{2}}|0\rangle+\frac{1}{\sqrt{2}}|1\rangle$
>
> $H|1\rangle=|-\rangle=\frac{1}{\sqrt{2}}|0\rangle-\frac{1}{\sqrt{2}}|1\rangle$

In [ ]:
#
# Let now Alice create a |+> qubit!
#
q = qubit(alice) # creates |0> by default
q.H() # the Hadamard gate: H|0>=|+>

#
# I'd like to see the amplitudes to be sure... CAN I HAVE A LOOK?
#
# NO WAY, SimulaQron doesn't let you cheat! You HAVE TO MEASURE if you want any information out of a qubit!
#

In [ ]:
#
# Alice measures q. This will return randomly 0 or 1, with 50% chance each.
# (Because in q, both |0> and |1> have absolute amplitude squared equal to 1/2.)
#
m = q.measure()
print("Measurement returned {}".format(m))

#
# In theory, this is a so-called True Random Number Generator (TRNG).
# (A quantum random number generator typically creates and measures |+> qubits under the hood.)
#

# Exercise: create and measure a $|-\rangle$ qubit
- Solution: double-click here.
<font color="white">
# prepare |-> in variable q
q = qubit(alice) # |0>
q.X() # changes q to |1>, as X|0>=|1>
q.H() # changes q to |->, as H|1>=|->
# measure q in variable m (again, it's 50-50% chance to get 0 or 1, can you explain why?)
m = q.measure()
</font>

In [ ]:
#
# Write your code below.
#

# prepare |-> in variable q
YOUR CODE COMES HERE
# measure q in variable m (again, it's 50-50% chance to get 0 or 1, can you explain why?)
YOUR CODE COMES HERE
print("Measurement returned {}".format(m))

#
# It's a typical mistake to accidentally prepare the |+> state instead of |->.
# Check the "official" solution above to make sure you really prepared the |-> state.
#
# Bonus question: what is X|+>?
# Hint: what's the amplitude vector of X|+>?
#

In [ ]:
#
# To prove that it's really 50-50% chance, place your previous code below.
# Here we create and measure 100 times the |-> qubit.
#
N = 100
count = 0

for i in range(0, N):
    PLACE YOUR CODE TO CREATE AND MEASURE |-> FROM ABOVE HERE
    USE INDENTATION OF 4 SPACES INSIDE THE FOR LOOP, LIKE THIS VERY LINE, OTHERWISE PYTHON COMPLAINS
    print(m, end=' ')
    count += m

print("\n\nNumber of 0s: {}\nNumber of 1s: {}".format(N-count, count))

In [ ]:
#
# But wait a moment! Didn't I say MAX. 20 QUBITS in the register? We've just created 100! How comes?
#
# Sure, we did, but each qubit was measured immediately after creation. So at no point in time were there more than 1 qubit in Alice's register.
#
# Let's see what Alice has now in her register.
#
num_qubits(alice)

# Alice encodes a bit as qubit and sends it to Bob
- Encoding a bit in the **Standard way**:
> $0\leftrightarrow|0\rangle$
>
> $1\leftrightarrow|1\rangle$


- Encoding a bit in the **Hadamard way**:
> $0\leftrightarrow|+\rangle$
>
> $1\leftrightarrow|-\rangle$

In [ ]:
#
# Alice sends 1 to Bob, encoded in the Hadamard way.
#
q = qubit(alice) # |0>
q.X() # |1>
q.H() # |-> (this corresponds to 1 in the Hadamard way of encoding)

num_qubits(alice)
print("Sending qubit to Bob...")
alice.sendQubit(q, "Bob") # qubit is gone, it's with Bob now
num_qubits(alice)

# Bob receives the qubit

In [ ]:
#
# Interface for Bob.
#
bob = CQCConnection("Bob")

In [ ]:
num_qubits(bob)
cleanup_qubits(bob)

In [ ]:
#
# Bob receives the qubit from Alice.
#
q = bob.recvQubit()

# But Bob doesn't know the way the qubit has been encoded by Alice!!!
> So he makes a guess, and decodes according to his guess. However, if he guesses it wrong, he may get the wrong result.
>
> For example, if Alice sent $|+\rangle$ but Bob guesses that Alice used the Standard way of encoding, then he will (wrongly) expect that either $q=|0\rangle$ or $q=|1\rangle$ holds. Thus, he will believe that by simply measuring $q$ he can retrieve the bit value that Alice sent (see code below). But in fact, he will measure $q=|+\rangle$ and get randomly 0 or 1, with 50% chance each.

In [ ]:
#
# Bob decodes the qubit into bit, according to his guess.
#
guess = "Standard" # we know Alice encoded in the Hadamard way, so Bob's guess is wrong

if guess == "Standard":
    dec = q.measure()
elif guess == "Hadamard":
    q.H() # brings |+> to |0>, |-> to |1>
    dec = q.measure()

print("Decoded bit value: {}".format(dec))

# Exercise:
> What if $q=|1\rangle$, but Bob guesses "Hadamard"? How much chance does he have to get 1 as decoded bit value?
- Solution: double-click here.
<font color="white">
    50% chance.
    So, in general, if Bob's guess is right, he'll retrieve exactly the bit value that Alice sent. However, if his guess is wrong, he has only 50% chance to decode successfully.
</font>

# Congratulations, well done!
- **Bonus**: If you still have time, continue with **Eve receives 100 qubits**.
- Or just check out the SimulaQron website:
> <a>http://www.simulaqron.org/</a>

# Eve receives 100 qubits

In [ ]:
from cqc.pythonLib import *
from random import randint

In [ ]:
#
# Interface for Eve.
#
eve = CQCConnection("Eve")

In [ ]:
num_qubits(eve)
cleanup_qubits(eve)

In [ ]:
#
# CASE 1
#
# Eve receives 100 qubits, one after the other. She knows each qubit is EITHER |0> OR |1>.
# When she measures, she gets 0 or 1, with 50% chance each.
#
N = 100
count = 0

for i in range(0, N):
    # randomly create |0> or |1>, and "deliver" it to Eve
    q = qubit(eve)
    if randint(0, 1) == 1:
        q.X() # flips |0> to |1>

    # Eve measures the received qubit q
    m = q.measure()
    print(m, end=' ')
    count += m

print("\n\nNumber of 0s: {}\nNumber of 1s: {}".format(N-count, count))

In [ ]:
#
# CASE 2
#
# Eve receives again 100 qubits, one after the other. This time she knows EVERY qubit is a |+>.
# Still, when she measures, she gets 0 or 1, with 50% chance each.
#
N = 100
count = 0

for i in range(0, N):
    # always create |+>, and "deliver" it to Eve
    q = qubit(eve)
    q.H() # |0> to |+>

    # Eve measures the received qubit q
    m = q.measure()
    print(m, end=' ')
    count += m

print("\n\nNumber of 0s: {}\nNumber of 1s: {}".format(N-count, count))

# Exercise: tell CASE 1 and CASE 2 apart
- Solution: double-click here.
<font color="white">
q.H() # changes |+> to |0>, as well as |0> to |+> and |1> to |->
</font>

In [ ]:
#
# PROBLEM: By simply measuring, Eve cannot tell CASE 1 and CASE 2 apart, because the statistics 
# are the same: about 50% 0s, and 50% 1s.
#
# However, Eve figured out that by transforming the received (unknown) qubit q, she can make the 
# statistics of CASE 1 and CASE 2 different!!! The moral of the story is that the superposition 
# state |+> is NOT the same as being either |0> or |1>, because there IS a way to distinguish 
# between the two cases.
#
# What does she have on her mind? Write your solution below where indicated.
#
N = 100

for case in range(1, 3):
    print("CASE {}".format(case))
    count = 0

    for i in range(0, N):
        if case == 1:
            # randomly create |0> or |1>, and "deliver" it to Eve
            q = qubit(eve)
            if randint(0, 1) == 1:
                q.X() # flips |0> to |1>
        else:
            # always create |+>, and "deliver" it to Eve
            q = qubit(eve)
            q.H() # |0> to |+>

        # Eve smartly transforms the received qubit q, by applying quantum gate(s).
        #
        # The point is that the SAME transformation should be applied to BOTH cases, as Eve 
        # doesn't know which case she is facing!!! (She has no access to the 'case' variable, 
        # that's exactly what she wants to figure out!)
        #
        PLACE YOUR CODE TO HERE TO TRANSFORM q
        USE INDENTATION OF 8 SPACES, OTHERWISE PYTHON COMPLAINS

        # Eve measures the transformed qubit q
        m = q.measure()
        print(m, end=' ')
        count += m

    print("\n\nNumber of 0s: {}\nNumber of 1s: {}\n".format(N-count, count))

# End of hands-on
- If you still have time, check out the SimulaQron website:
> <a>http://www.simulaqron.org/</a>